# Log normal for stock price

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed()

## Stats from price

The IBM stock price was downloaded from [Google Finance](https://www.google.com/finance?q=NYSE%3AIBM&ei=yoGkWJiPIszEU9rvtfAB) on 15 Feb 2017.

In [ ]:
ibm = pd.read_csv('ibm.csv')
ibm['Date'] = pd.to_datetime(ibm['Date'])
ibm.set_index('Date', inplace=True, verify_integrity=True)
ibm.sort_index(inplace=True)

Let $r_i$ be the rate of return given stock prices $S_i$ and $S_{i-1}$ for time $t_i$ and $t_{i-1}$, *e.g.*,
$$ r_i = \frac{S_i - S_{i-1}}{S_{i-1}}. $$
If we assume the price of the stock is log normally distributed, then $\log(1+r_i)$ is normally distributed since
$$ 1+r_i = \frac{S_i}{S_{i-1}} = e^{\log\left( \frac{S_i}{S_{i-1}} \right)}. $$
(Note that in practice, $1+r_i$ is close to 1 most of the time, so $\log(1+r_i)$ is close to zero, which means $\log(1+r_i)$ is close to $r_i$ most of the time.)

For $n$ stock price data points, let $Y_i = \log(S_i/S_{i-1})$ where $i\in[0,n)$. To compute the drift (mean) and volatility (standard deviation) from the data, we use the unbiased estimators
$$ \mu = E[Y] $$
and
$$ \sigma^2 = \frac{n}{n-1}\left( E[Y^2] - \mu^2 \right). $$

In [ ]:
Y = np.diff(np.log(ibm['Close']))
n = Y.size
mu = Y.mean()
sigma = np.sqrt(((Y**2.).mean() - mu**2)*(n/(n-1)))
print('mu = ', mu)
print('sigma = ', sigma)

To check our variables, we plot the stock price vs. $e^{i\mu}$ for $i\in[0,n)$.

In [ ]:
ibm.plot(y='Close')
plt.plot(ibm.index, ibm['Close'][0]*np.exp(np.arange(0, n+1)*mu))
plt.show()

We can also see that most of the returns lie within $2\sigma$ of $\mu$, which matches the empirical rule which says 95% of observations fall within $2\sigma$ of the mean.

In [ ]:
xlim = [0, n-1]
plt.plot([0, n-1], [mu+2*sigma, mu+2*sigma], color='tab:green')
plt.plot([0, n-1], [mu-2*sigma, mu-2*sigma], color='tab:green')
plt.plot([0, n-1], [mu, mu], color='tab:red')
plt.xlim([0, n-1])
plt.plot(Y, marker='.', ls='', color='tab:blue')
plt.show()

## Simulating price

Recall that if we define $X = e^{\sigma Z + \mu}$ for $Z\sim N(0, 1)$, then $X\sim N(\mu, \sigma^2)$.

So the model above suggests that if we consider price $S_{i-1}$, then price $S_i$ is modeled by
$$ S_i = S_{i-1} X. $$
Given the initial price $S_0$, this means we can find any $S_i$ given a sequence of *i.i.d.* variables $X_i = e^{\sigma Z_i + \mu}$ for *i.i.d.* $Z_i\sim N(0, 1)$ by
$$ S_i = S_0 \prod_{k=1}^i X_i. $$

In [ ]:
def gbm(S0, mu, sigma, n):
    X = np.exp(sigma*np.random.normal(0, 1, n) + mu)
    X = np.insert(X, 0, 1.)
    return S0*np.cumprod(X)

To compare the simulated price and the actual price, we run 10 simulations and plot them in blue against the actual price in black. It becomes quickly clear the strength and weakness of this price model.

In [ ]:
ibm.plot(y='Close', color='k')
S0 = ibm['Close'][0]
for _ in range(0, 10):
    plt.plot(ibm.index, gbm(S0, mu, sigma, n), 'tab:cyan', zorder=1)
plt.show()

## References

[Lognormal model for stock prices](http://www.math.ucsd.edu/~msharpe/stockgrowth.pdf)

## Appendix
### Quant Start point of view

This site seems like it has some information, but the very basic stuff is scattered. [Time series analysys](https://www.quantstart.com/articles/Beginners-Guide-to-Time-Series-Analysis) lists some articles (that seem yet to be written), one including a series on stochastic models, which include
[geometric Brownian motion](https://www.quantstart.com/articles/Geometric-Brownian-Motion).
This seems to be derived from the
[stochastic differential equation](https://www.quantstart.com/articles/Stochastic-Differential-Equations)
which mention
[Brownian motion and the Weiner process](https://www.quantstart.com/articles/Brownian-Motion-and-the-Wiener-Process)
as a prerequisite. This is after mentioning a
[rationale for stochastic calculus](https://www.quantstart.com/articles/Introduction-to-Stochastic-Calculus)
(which I haven't chased down yet), and definitions of the
[Markov and Martingale properties](https://www.quantstart.com/articles/The-Markov-and-Martingale-Properties),
which I also have not looked at.